In [1]:
from pprint import pprint
import json
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [2]:
def apd_salary(i):
    
    mi, ma = '', ''
    result = ['','','']
    
    i = i.replace(u'\xa0', u'')
    i = i.replace(u'.', u'')
    
    if re.search('от ', i):
        min = re.findall('\d+', i)
        mi = min[0]
        
    if re.search('до ', i):
        max = re.findall('\d+', i)
        ma = max[0]
 
    if re.search('\d+-\d+', i):
        min = re.findall('\d+', i)
        max = re.findall('-\d+', i)
        max[0] = max[0].replace(u'-', u'')
        mi = min[0]
        ma = max[0]
        
    result[0] = mi
    result[1] = ma
    result[2] = i[-3:]
    
    return(result) 

In [3]:
def parsing(html, site):
    res_block = html.find('div',{'class':'vacancy-serp vacancy-serp_xs-mode'})
    res_list = res_block.findChildren(recursive=False)

    results=[]
    for vak in res_list:
        data = {}
        vak_salary = ['','','']
        try:    
            vak_name = vak.find('a',{'class':'bloko-link HH-LinkModifier'}).text#.findChild()
            vak_link = vak.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        except:
            continue           
    
        try:
            vak_s = vak.find('div',{'class':'vacancy-serp-item__compensation'}).getText()
            vak_salary = apd_salary(vak_s)
        except:
            pass

        data['name'] = vak_name
        data['salary_min'] = vak_salary[0]
        data['salary_max'] = vak_salary[1]
        data['salary_valuta'] = vak_salary[2]
        data['link'] = vak_link
        data['site'] = site

        results.append(data)    
        
    return(results)

main_link = 'https://hh.ru'
#user_input = input()
link = main_link + '/search/vacancy?L_save_area=true&clusters=true&enable_snippets=true&search_field=name&text=курьер&showClusters=false'#+user_input
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
# &page=8
response = requests.get(link, headers=headers).text
html = bs(response,'lxml')

#pprint(response)
#html

res_block = html.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})

# СЧИТАЕМ СКРОЛЛИНГ
scr = html.findAll('a',{'class':'bloko-button HH-Pager-Control'})
scr = int(scr[-1].text)

i = 1
results=[]
while res_block:
    res_block = html.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})
    print(int(100*i/(scr+1)), ' %')
    results += parsing(html, main_link)
    link1 = link + '&page=' + str(i)
    i += 1
    response = requests.get(link1, headers=headers).text
    html = bs(response,'lxml')

# pprint(results)

with open('data.json', 'w') as outfile:
    json.dump(results, outfile)

print('DONE!')

2  %


AttributeError: 'NoneType' object has no attribute 'findChildren'

In [4]:
with open('data.json') as fh: 
    data = json.load(fh)
len(data)
#data

FileNotFoundError: [Errno 2] No such file or directory: 'data.json'

In [5]:
df1 = pd.DataFrame(data)
df1.head(25)

NameError: name 'data' is not defined